# Build master data + feature-rich model
Single notebook to build master_data and engineer richer features for EC/pH/soil temp forecasting.

In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
from datetime import datetime, timedelta, time
import csv
from typing import Any, Dict, List, Optional
from openpyxl import load_workbook
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt


In [10]:
# --------------------
# Data build utilities
# --------------------

def floor_to_10min(dt: datetime) -> datetime:
    minute = (dt.minute // 10) * 10
    return dt.replace(minute=minute, second=0, microsecond=0)

def parse_time_flex(text: str) -> time:
    text = text.strip()
    parts = text.split(":")
    if len(parts) >= 2:
        try:
            h = int(parts[0]); m = int(parts[1]); s = int(parts[2]) if len(parts) >= 3 and parts[2] else 0
            return time(hour=h, minute=m, second=s)
        except Exception:
            pass
    for fmt in ("%H:%M:%S", "%H:%M"):
        try:
            return datetime.strptime(text, fmt).time()
        except Exception:
            continue
    raise ValueError(f"Unrecognized time format: {text}")

def parse_ts_variant(text: str, day_first: bool) -> datetime:
    norm = text.replace("-", "/").replace(".", "/").strip()
    if " " not in norm:
        raise ValueError("No space between date and time")
    date_part, time_part = norm.split(" ", 1)
    t_val = parse_time_flex(time_part)
    parts = date_part.split("/")
    if len(parts) < 3:
        raise ValueError("Date part incomplete")
    a, b, y = parts[0], parts[1], parts[2]
    if len(y) == 2:
        y = "20" + y
    if day_first:
        day, month = int(a), int(b)
    else:
        month, day = int(a), int(b)
    return datetime(int(y), month, day, t_val.hour, t_val.minute, t_val.second)

def parse_micro_ts(value: Any, prev_ts: Optional[datetime]) -> datetime:
    if isinstance(value, datetime):
        ts = value
    else:
        text = str(value).strip()
        candidates = []
        for day_first in (True, False):
            try:
                candidates.append(parse_ts_variant(text, day_first))
            except Exception:
                continue
        if not candidates:
            raise ValueError(f"Unrecognized micro-climate timestamp: {value}")
        if prev_ts is not None:
            best = None; best_delta = None
            for dt in candidates:
                delta = dt - prev_ts
                if delta >= timedelta(minutes=-5) and (best_delta is None or delta < best_delta):
                    best = dt; best_delta = delta
            ts = best if best is not None else candidates[0]
        else:
            ts = candidates[0]
    return ts

def parse_time_robust(val: Any) -> time:
    if isinstance(val, datetime):
        return val.time()
    if isinstance(val, time):
        return val
    return parse_time_flex(str(val))

def parse_date_dayfirst(text: str) -> datetime.date:
    text = text.strip()
    norm = text.replace("-", "/").replace(".", "/")
    parts = norm.split("/")
    if len(parts) >= 3:
        d, m, y = parts[0], parts[1], parts[2]
        if len(y) == 2:
            y = "20" + y
        return datetime(int(y), int(m), int(d)).date()
    for fmt in ("%d/%m/%Y", "%d/%m/%y", "%Y-%m-%d"):
        try:
            return datetime.strptime(text, fmt).date()
        except Exception:
            continue
    raise ValueError(f"Unrecognized date format: {text}")

def read_micro(path: str) -> List[Dict[str, Any]]:
    wb = load_workbook(path, read_only=True, data_only=True)
    ws = wb["micro_climate_rh_t_et0"]
    rows = ws.iter_rows(values_only=True)
    next(rows)
    data = []
    prev_ts = None
    base_ts = None
    interval = timedelta(minutes=10)
    for row in rows:
        if not row or row[0] is None:
            continue
        raw_ts = parse_micro_ts(row[0], prev_ts)
        if base_ts is None:
            base_ts = raw_ts
            seq_ts = base_ts
        else:
            seq_ts = base_ts + interval * len(data)
        prev_ts = raw_ts
        data.append({
            "timestamp": seq_ts,
            "ET0": row[1],
            "internal_air_temp_c": row[2],
            "internal_rh_%": row[3],
            "internal_radiation": row[4],
        })
    return data

def read_irrigation(path: str):
    wb = load_workbook(path, read_only=True, data_only=True)
    ws = wb.active
    rows = ws.iter_rows(values_only=True)
    next(rows)
    events = []
    for row in rows:
        if not row or row[0] is None:
            continue
        date_val, time_val, irr_ml, fert_type = row[0], row[1], row[2], row[3]
        if isinstance(date_val, datetime):
            date_only = date_val.date()
        else:
            parsed = None
            for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%d.%m.%Y", "%m.%d.%Y"):
                try:
                    parsed = datetime.strptime(str(date_val), fmt).date(); break
                except Exception:
                    continue
            if parsed is None:
                parsed = parse_date_dayfirst(str(date_val))
            date_only = parsed
        t_val = parse_time_robust(time_val)
        dt = datetime.combine(date_only, t_val)
        bucket = floor_to_10min(dt)
        fert_code = 0
        if isinstance(fert_type, str):
            f = fert_type.strip().upper()
            if f == "A":
                fert_code = 1
            elif f == "B":
                fert_code = 2
        events.append({"bucket": bucket, "irr_ml": irr_ml or 0.0, "fert_code": fert_code})
    return events

def build_irrigation_series(events):
    fert_by_bucket = {}
    irr_by_bucket = defaultdict(float)
    for ev in events:
        irr_by_bucket[ev["bucket"]] += ev["irr_ml"]
        if ev["fert_code"]:
            fert_by_bucket[ev["bucket"]] = max(fert_by_bucket.get(ev["bucket"], 0), ev["fert_code"])
    return irr_by_bucket, fert_by_bucket

def parse_ph_ec_date(val: Any) -> Optional[datetime.date]:
    if isinstance(val, datetime):
        return val.date()
    if val is None:
        return None
    s = str(val).strip()
    if not s:
        return None
    for sep in (".", "/", "-"):
        if sep in s:
            parts = s.replace("-", sep).replace("/", sep).split(sep)
            if len(parts) >= 3:
                d_raw, m_raw, y_raw = parts[0], parts[1], parts[2]
                for day_first in (True, False):
                    day, month = (d_raw, m_raw) if day_first else (m_raw, d_raw)
                    year = y_raw
                    if len(year) == 2:
                        year = "20" + year
                    try:
                        return datetime(int(year), int(month), int(day)).date()
                    except Exception:
                        continue
    for fmt in ("%Y-%m-%d", "%Y/%m/%d", "%m.%d.%Y", "%d.%m.%Y", "%m-%d-%Y", "%d-%m-%Y"):
        try:
            return datetime.strptime(s, fmt).date()
        except Exception:
            continue
    return None

def read_ph_ec(path: str):
    wb = load_workbook(path, read_only=True, data_only=True)
    ws = wb.active
    rows = ws.iter_rows(values_only=True)
    next(rows)
    measurements = []
    for row in rows:
        if not row or row[1] is None:
            continue
        date_part = parse_ph_ec_date(row[1])
        time_part = row[4]
        if date_part is None or time_part is None:
            continue
        t_val = parse_time_robust(time_part)
        dt = datetime.combine(date_part, t_val)
        measurements.append({"timestamp": dt, "ph": row[2], "ec_ms": row[3]})
    return measurements

def map_ph_ec_to_micro(timestamps: List[datetime], measurements: List[Dict[str, Any]]):
    ph_list = [None] * len(timestamps)
    ec_list = [None] * len(timestamps)
    for m in measurements:
        best_idx = None; best_diff = None
        for idx, ts in enumerate(timestamps):
            diff = abs(ts - m["timestamp"])
            if diff <= timedelta(minutes=10) and (best_diff is None or diff < best_diff):
                best_idx = idx; best_diff = diff
        if best_idx is not None:
            ph_list[best_idx] = m["ph"]
            ec_list[best_idx] = m["ec_ms"]
    return ph_list, ec_list

def read_soil_temp(path: str):
    data = {}
    with open(path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            ts = datetime.strptime(row["datetime"], "%Y-%m-%d %H:%M:%S")
            data[ts] = float(row["soil_temp_pred"]) if row["soil_temp_pred"] != "" else None
    return data

def read_canopy(path: str):
    wb = load_workbook(path, read_only=True, data_only=True)
    ws = wb.active
    rows = ws.iter_rows(values_only=True)
    next(rows)
    canopy = {}
    for row in rows:
        if len(row) < 7:
            continue
        date_val = row[4]
        canopy_val = row[6]
        if date_val is None:
            continue
        if not isinstance(date_val, datetime):
            try:
                date_val = datetime.strptime(str(date_val), "%Y-%m-%d")
            except Exception:
                continue
        canopy[date_val.date()] = canopy_val
    return canopy

def build_master():
    micro = read_micro("micro_climate_rh_t_et0.xlsx")
    events = read_irrigation("Irrigation + ALL Elemental Fractions schedule for one plant (100N).xlsx")
    irr_by_bucket, fert_by_bucket = build_irrigation_series(events)
    ph_ec = read_ph_ec("PH+EC Final.xlsx")
    soil_temp = read_soil_temp("soil_temp_predictions_full_range.csv")
    canopy = read_canopy("Daily Canopy Cover Values.xlsx")

    timestamps = [row["timestamp"] for row in micro]
    ph_list, ec_list = map_ph_ec_to_micro(timestamps, ph_ec)

    irr_values = []
    irrigation_3h = []
    fert_flags = []
    soil_list = []
    canopy_list = []
    irr_current = []

    for i, ts in enumerate(timestamps):
        irr_now = irr_by_bucket.get(ts, 0.0)
        irr_current.append(irr_now)
        irr_values.append(irr_now)
        window_sum = sum(irr_values[max(0, i - 17): i + 1])
        irrigation_3h.append(window_sum)
        fert_flags.append(fert_by_bucket.get(ts, 0))
        soil_list.append(soil_temp.get(ts))
        canopy_list.append(canopy.get(ts.date()))

    df_out = pd.DataFrame({
        "timestamp": timestamps,
        "ET0": [r["ET0"] for r in micro],
        "internal_air_temp_c": [r["internal_air_temp_c"] for r in micro],
        "internal_rh_%": [r["internal_rh_%"] for r in micro],
        "internal_radiation": [r["internal_radiation"] for r in micro],
        "irrigation_ml_current": irr_current,
        "irrigation_last_3h_ml": irrigation_3h,
        "fertilization_flag": fert_flags,
        "ph": ph_list,
        "ec_ms": ec_list,
        "soil_temp_pred": soil_list,
        "canopy_cover": canopy_list,
    })
    df_out = df_out.sort_values("timestamp").set_index("timestamp")
    out_path = Path("master_data.csv")
    df_out.to_csv(out_path, index=True)
    print(f"Wrote {out_path} with {len(df_out)} rows")
    return df_out

print("Building master_data ...")
df = build_master()


Building master_data ...
Wrote master_data.csv with 16682 rows


In [11]:
# 2b) Build ds from master_data anchor-to-anchor intervals
master = pd.read_csv('master_data.csv', parse_dates=['timestamp'])
master = master.sort_values('timestamp')
anchors = master[master['ph'].notna() & master['ec_ms'].notna()].copy()
anchors['t1'] = anchors['timestamp'].shift(-1)
anchors['ph1'] = anchors['ph'].shift(-1)
anchors['ec1'] = anchors['ec_ms'].shift(-1)
anchors['gap_hours'] = (anchors['t1'] - anchors['timestamp']).dt.total_seconds()/3600
anchors['dph'] = anchors['ph1'] - anchors['ph']
anchors['dec'] = anchors['ec1'] - anchors['ec_ms']
# drop last anchor without next
anchors = anchors.dropna(subset=['t1','ph1','ec1','dph','dec'])
# Feature set: current obs and context
ds = anchors[[
    'timestamp','t1','gap_hours',
    'ph','ec_ms','soil_temp_pred','internal_air_temp_c','internal_rh_%',
    'internal_radiation','ET0','irrigation_last_3h_ml','fertilization_flag','canopy_cover',
    'dph','dec'
]].copy()
ds = ds.rename(columns={'ph':'ph0','ec_ms':'ec0'})
ds = ds.set_index('timestamp')
print('Built ds with intervals:', len(ds))


Built ds with intervals: 108


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

def rmse(a, b):
    return float(np.sqrt(mean_squared_error(a, b)))

def relative_error_percent(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), eps) * 100

# ============================================================
# 4) Walk-forward (test every interval, no skips)
# ============================================================
feature_cols = [c for c in ds.columns if c not in ("dph","dec","t1")]
X_all = ds[feature_cols].copy()
y_all = ds[["dph","dec"]].copy()

initial_train = 30

if len(ds) <= initial_train + 1:
    raise RuntimeError(f"Not enough intervals: n={len(ds)} for initial_train={initial_train}")

pred_rows = []

# Loop over every next interval from initial_train to end-1
for i in range(initial_train, len(ds)):
    X_train = X_all.iloc[:i]
    y_train = y_all.iloc[:i]

    X_test  = X_all.iloc[i:i+1]
    y_test  = y_all.iloc[i:i+1]

    base_rf = RandomForestRegressor(
        n_estimators=800,
        max_depth=8,
        min_samples_leaf=4,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1
    )
    model = MultiOutputRegressor(base_rf)
    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    dph_pred, dec_pred = float(pred[0, 0]), float(pred[0, 1])

    ph0 = float(ds.iloc[i]["ph0"])
    ec0 = float(ds.iloc[i]["ec0"])

    dph_true = float(y_test["dph"].iloc[0])
    dec_true = float(y_test["dec"].iloc[0])

    ph1_true = ph0 + dph_true
    ec1_true = ec0 + dec_true

    ph1_pred = ph0 + dph_pred
    ec1_pred = ec0 + dec_pred

    pred_rows.append({
        "t0": ds.index[i],
        "t1": ds.iloc[i]["t1"],
        "gap_hours": float(ds.iloc[i]["gap_hours"]),
        "ph0": ph0, "ec0": ec0,
        "dph_true": dph_true, "dec_true": dec_true,
        "dph_pred": dph_pred, "dec_pred": dec_pred,
        "ph1_true": ph1_true, "ec1_true": ec1_true,
        "ph1_pred": ph1_pred, "ec1_pred": ec1_pred,
        "train_intervals": i
    })

pred_df = pd.DataFrame(pred_rows).set_index("t0").sort_index()

print("Tested intervals (no skips):", len(pred_df), "out of total:", len(ds),
      "| first test index:", initial_train)

# ============================================================
# 5) Overall metrics on ALL tested intervals
# ============================================================
ph1_true = pred_df["ph1_true"].values
ph1_pred = pred_df["ph1_pred"].values
ec1_true = pred_df["ec1_true"].values
ec1_pred = pred_df["ec1_pred"].values

print("\n=== Overall next-observation performance (ALL tested intervals) ===")
print("PH MAE:", float(mean_absolute_error(ph1_true, ph1_pred)),
      "RMSE:", rmse(ph1_true, ph1_pred),
      "R2:", float(r2_score(ph1_true, ph1_pred)))
print("EC MAE:", float(mean_absolute_error(ec1_true, ec1_pred)),
      "RMSE:", rmse(ec1_true, ec1_pred),
      "R2:", float(r2_score(ec1_true, ec1_pred)))

# Relative error
ph_rel = relative_error_percent(ph1_true, ph1_pred)
ec_rel = relative_error_percent(ec1_true, ec1_pred)
print("\n=== Relative Error (%) (ALL tested intervals) ===")
print("PH Mean RE (%):", float(ph_rel.mean()), "Median RE (%):", float(np.median(ph_rel)))
print("EC Mean RE (%):", float(ec_rel.mean()), "Median RE (%):", float(np.median(ec_rel)))

# ============================================================
# 6) EC range breakdown
# ============================================================
ec_bins = pd.cut(
    pred_df["ec1_true"],
    bins=[0, 0.3, 1.0, 3.0, 10],
    labels=["<0.3", "0.3–1", "1–3", ">3"]
)
print("\nEC range counts (tested):")
print(ec_bins.value_counts(dropna=False))

for b in ec_bins.dropna().unique():
    mask = (ec_bins == b).values
    print(f"EC range {b}: n={mask.sum()} Mean RE (%): {ec_rel[mask].mean():.3f} Median RE (%): {np.median(ec_rel[mask]):.3f}")

Tested intervals (no skips): 78 out of total: 108 | first test index: 30

=== Overall next-observation performance (ALL tested intervals) ===
PH MAE: 0.7234278396392958 RMSE: 0.9270640628418901 R2: 0.5792511094588813
EC MAE: 0.3010996305631767 RMSE: 0.5839816139274692 R2: 0.7543452071085192

=== Relative Error (%) (ALL tested intervals) ===
PH Mean RE (%): 8.845067982498154 Median RE (%): 6.881928666722287
EC Mean RE (%): 64.3659351516853 Median RE (%): 29.190954827521047

EC range counts (tested):
ec1_true
0.3–1    41
<0.3     24
1–3       7
>3        6
Name: count, dtype: int64
EC range 1–3: n=7 Mean RE (%): 21.210 Median RE (%): 24.146
EC range 0.3–1: n=41 Mean RE (%): 46.058 Median RE (%): 19.946
EC range <0.3: n=24 Mean RE (%): 117.556 Median RE (%): 73.082
EC range >3: n=6 Mean RE (%): 27.058 Median RE (%): 16.713


In [13]:
# ============================================================
# 7) Save predictions
# ============================================================
pred_df.to_csv("rf_rootzone_model.csv")
print("\nSaved: rf_rootzone_model.csv")


Saved: rf_rootzone_model.csv
